In [8]:
import os

In [9]:
%pwd

'C:\\ML Projects\\Facial_Emotion_Recognition_End2End'

In [10]:
os.chdir('C:\\ML Projects\Facial_Emotion_Recognition_End2End')

In [11]:
%pwd

'C:\\ML Projects\\Facial_Emotion_Recognition_End2End'

In [12]:
from dataclasses import dataclass
from pathlib import Path

In [13]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir :Path


In [24]:
from src.Facial_Emotion_Recognition.constant import *
from src.Facial_Emotion_Recognition.utils.common import read_yaml,create_directories
# from src.Facial_Emotion_Recognition.constant import *

In [25]:
class ConfigurationManager:

    # This class is used to manage the configuration of the application.
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #to get values by calling key and then it will create directories

    def get_data_ingetsion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion  #calling config.yaml values by key 

        create_directories([config.root_dir]) # also this 

        data_ingestion_config=DataIngestionConfig(

            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config
    
    

In [26]:
import os
import urllib.request as request
import zipfile
from src.Facial_Emotion_Recognition.logger import logger
from src.Facial_Emotion_Recognition.utils.common import get_size

In [33]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config= config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename ,header =request.urlretrieve(

                url=self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with following info: {header}')

        else:
            logger.info(f'file already exists of size :{get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):

        unzip_path= self.config.unzip_dir

        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_file:
            zip_file.extractall(unzip_path)
        


In [34]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingetsion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2023-10-13 16:04:36,092: INFO: common: yaml file : config\config.yaml loaded successfully]
[2023-10-13 16:04:36,098: INFO: common: yaml file : params.yaml loaded successfully]
[2023-10-13 16:04:36,101: INFO: common: created directories at :artifacts]
[2023-10-13 16:04:36,106: INFO: common: created directories at :artifacts/data_ingestion]
[2023-10-13 16:05:17,338: INFO: 1944526108: artifacts/data_ingestion/data.zip downloaded with following info: Connection: close
Content-Length: 61313305
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e2350693c4419684f6fe3b245c313dc3c667eb82f19d5f0ab62877f26c4f876a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1142:5DD7F:6DB1B:B1EA6:65291D3A
Accept-Ranges: bytes
Date: Fri, 13 Oct 2023 10:34:43 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100033-HYD
X-Ca